# Egyszerű Képosztályozó

Ez a jegyzetfüzet bemutatja, hogyan lehet képeket osztályozni egy előre betanított neurális hálózat segítségével.

**Amit megtanulsz:**
- Hogyan tölts be és használj egy előre betanított modellt
- Képek előfeldolgozása
- Előrejelzések készítése képeken
- A bizalmi pontszámok megértése

**Felhasználási eset:** Tárgyak azonosítása képeken (például "macska", "kutya", "autó" stb.)

---


## 1. lépés: Szükséges könyvtárak importálása

Importáljuk azokat az eszközöket, amelyekre szükségünk lesz. Ne aggódj, ha még nem értesz mindent!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## 2. lépés: Előre betanított modell betöltése

A **MobileNetV2** modellt fogjuk használni, amelyet már több millió képen betanítottak.

Ezt **Transfer Learningnek** nevezzük – egy más által betanított modell használata!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## 3. lépés: Segédfüggvények

Hozzunk létre függvényeket a képek betöltésére és előkészítésére a modellünk számára.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## 4. lépés: Tesztelés mintaképeken

Próbáljunk meg néhány képet osztályozni az internetről!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Osztályozd mindegyik képet


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## 5. lépés: Próbáld ki saját képeiddel!

Cseréld ki az alábbi URL-t bármilyen kép URL-re, amelyet szeretnél osztályozni.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Mi történt?

1. **Betöltöttünk egy előre betanított modellt** - A MobileNetV2 milliónyi képen lett betanítva
2. **Előfeldolgoztuk a képeket** - Átméreteztük és a modell számára megfelelő formátumba alakítottuk őket
3. **A modell előrejelzéseket készített** - Valószínűségeket adott 1000 objektumkategóriára
4. **Dekódoltuk az eredményeket** - A számokat ember által érthető címkékké alakítottuk

### A bizalmi pontszámok megértése

- **90-100%**: Nagyon magabiztos (szinte biztosan helyes)
- **70-90%**: Magabiztos (valószínűleg helyes)
- **50-70%**: Mérsékelten magabiztos (lehet, hogy helyes)
- **50% alatt**: Nem túl magabiztos (bizonytalan)

### Miért lehetnek hibásak az előrejelzések?

- **Szokatlan szög vagy megvilágítás** - A modellt tipikus fotókon tanították
- **Több objektum** - A modell egy fő objektumot vár
- **Ritka objektumok** - A modell csak 1000 kategóriát ismer
- **Gyenge minőségű kép** - Homályos vagy pixeles képeket nehezebb felismerni

---


## 🚀 Következő lépések

1. **Próbálj ki különböző képeket:**
   - Keress képeket az [Unsplash](https://unsplash.com) oldalon
   - Jobb kattintás → „Kép címének másolása” a URL megszerzéséhez

2. **Kísérletezz:**
   - Mi történik absztrakt művészettel?
   - Felismeri az objektumokat különböző szögekből?
   - Hogyan kezeli a több objektumot?

3. **Tanulj többet:**
   - Fedezd fel a [Számítógépes látás leckéket](../lessons/4-ComputerVision/README.md)
   - Tanuld meg, hogyan képezhetsz saját képosztályozót
   - Értsd meg, hogyan működnek a CNN-ek (Konvolúciós Neurális Hálózatok)

---

## 🎉 Gratulálunk!

Most építettél egy képosztályozót egy korszerű neurális hálózat segítségével!

Ugyanez a technika működteti:
- Google Photos (a fotóid rendszerezése)
- Önjáró autók (objektumok felismerése)
- Orvosi diagnosztika (röntgenképek elemzése)
- Minőségellenőrzés (hibák észlelése)

Folytasd a felfedezést és tanulást! 🚀



---

**Felelősség kizárása**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével került lefordításra. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
